In [0]:
from pyspark.sql.functions import sum, count, desc, approx_count_distinct

In [0]:
df_silver = spark.readStream.table('hj_orders.silver.orders_processed')

In [0]:
display(df_silver.printSchema())

In [0]:
%sql
USE CATALOG hj_orders;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS gold
MANAGED LOCATION 'abfss://transformed@himalayanjavadl.dfs.core.windows.net/gold/'

In [0]:
%sql
CREATE EXTERNAL VOLUME IF NOT EXISTS gold.checkpoints
LOCATION 'abfss://transformed@himalayanjavadl.dfs.core.windows.net/gold/checkpoints/transformed_folder'

In [0]:
# branch with the highest revenue
df_branch = df_silver.groupBy('branch') \
            .agg(sum('line_item_amount').alias('total_revenue'),
                  count('order_id').alias('total_number_of_items'),
                    approx_count_distinct('order_id').alias('total_number_of_orders')) \
                    .orderBy(desc('total_revenue'))

checkpoint_gold_branch = '/Volumes/hj_orders/gold/checkpoints/branch_sales'

query_branch = (df_branch.writeStream
                .format('delta')
                .outputMode('complete')
                .option('checkpointLocation', checkpoint_gold_branch)
                .toTable('hj_orders.gold.branch_sales_table'))


In [0]:
display(df_branch)

In [0]:
# highest selling product
df_product = df_silver.groupBy('item_id', 'item_name') \
            .agg(sum('quantity').alias('total_quantity'),
                  sum('line_item_amount').alias('total_revenue'),
                  count('order_id').alias('total_number_of_orders')) \
            .orderBy(desc('total_revenue'))

In [0]:
display(df_product)

In [0]:
checkpoint_gold_product = '/Volumes/hj_orders/gold/checkpoints/product_sales'

query_product = (df_product.writeStream
                 .format('delta')
                 .outputMode('complete')
                 .option('checkpointLocation', checkpoint_gold_product)
                 .toTable('hj_orders.gold.product_sales_table'))

In [0]:
%sql
DESCRIBE DETAIL hj_orders.gold.product_sales_table;

In [0]:
from pyspark.sql import functions as F

# 1. Define a Watermark (tells Spark how long to wait for late data)
# 2. Group by 1-hour windows
df_peak_hours = (df_silver
    .withWatermark("timestamp", "3 minutes") 
    .groupBy(F.window("timestamp", "1 hour"), "branch")
    .agg(F.sum("line_item_amount").alias("hourly_revenue"))
    .select("window.start", "window.end", "branch", "hourly_revenue")
    .orderBy("start"))

# Write to Gold Table
(df_peak_hours.writeStream
    .format("delta")
    .outputMode("complete")
    .option("checkpointLocation", "/Volumes/hj_orders/gold/checkpoints/peak_hours")
    .toTable("hj_orders.gold.hourly_sales_trends"))

In [0]:
display(df_peak_hours)

In [0]:
from pyspark.sql import functions as F

# Average order of value
df_aov = df_silver.groupBy('branch') \
            .agg(F.sum('line_item_amount').alias('total_order_amount'),
                 F.approx_count_distinct('order_id').alias('total_unique_orders')
                 ) \
        .withColumn("avg_order_value", F.round(F.col("total_order_amount") / F.col("total_unique_orders"), 2))

In [0]:
display(df_aov)

In [0]:
(df_aov.writeStream
 .format("delta")
 .outputMode("complete")
 .option("checkpointLocation", "/Volumes/hj_orders/gold/checkpoints/aov")
 .toTable("hj_orders.gold.average_order_value"))

In [0]:
%sql
DESCRIBE CATALOG EXTENDED hj_orders;